In [54]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from unidecode import unidecode
import pickle
import requests

nltk.download('stopwords')
nltk.download('punkt')

df = pd.read_csv('/root/projects/personal/comitiva_esperanca/data/ingestion/RAW/noticias_cepea.csv', sep=',', on_bad_lines='skip', encoding='utf-8')
df = df[df['data'] != 'data']
df = df.dropna()
df
padrao_data_cepea = r"Cepea, \d{2}/\d{2}/\d{4} - "
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_data_cepea, '', x))

## remover a palavra 'cepea' das noticias
padrao_cepea = r"Cepea"
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_cepea, '', x, flags=re.IGNORECASE))

## remover numeros das noticias
padrao_numeros = r'[0-9]+'
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_numeros, '', x))

## noticia que contem a palavra 'soja'
df = df[df['titulo'].str.contains('soja', flags=re.IGNORECASE)]

df


# Pré-processamento dos dados
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    # remover acentuação
    text = unidecode(text)
    # Remover pontuações
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenização
    words = word_tokenize(text.lower())
    # Remover stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['noticia'] = df['noticia'].apply(preprocess_text)

df_noticias = df.copy()

# Vetorização
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['noticia'])

# Carregar o modelo 

model = pickle.load(open('/root/projects/personal/comitiva_esperanca/jupyter/notebooks/playground/maycon/modelagem/modelo.pkl', 'rb'))

df_noticias = df_noticias.dropna(subset=['noticia']).reset_index(drop=True)
y_pred = model.predict_proba(df_noticias['noticia'])
y_pred_class = model.predict(df_noticias['noticia'])

## eliminar o e+01 do valor numerico em pandas
pd.options.display.float_format = '{:.2f}'.format

df_predict = pd.DataFrame(y_pred, columns=model.classes_)
df_predict['predict_final'] = df_predict.idxmax(axis=1)

df_pred_concatenado = pd.concat([df_noticias, df_predict], axis=1)
df_pred_concatenado[['data', 'titulo', 'noticia', 'predict_final', 'Neutra', 'Negativa', 'Positiva']] #.to_excel('validacao.xlsx', index=False)


## print group by predict_final
df_qtde_noticias = df_pred_concatenado.groupby('predict_final').count()['titulo']
print("="*10, "\nQuantidade de notícias por sentimento", df_qtde_noticias,"\n", "="*10)

## publicar na api 

def publicar_noticia(row):
    url = 'http://20.241.232.187/news'
    data = {
        'title': row['titulo'],
        'newsContent': row['noticia'],
        "commodityType": "Soja",
        "source": "CEPEA",
        "positiveSentiment": row['Positiva'],
        "neutralSentiment": row['Neutra'],
        "negativeSentiment": row['Negativa'],
        "finalSentiment": row['predict_final'],
    }
    response = requests.post(url, json=data)
    print(response.status_code, response.text)

df_pred_concatenado.apply(publicar_noticia, axis=1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/root/projects/personal/comitiva_esperanca/.python_env/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/root/projects/personal/comitiva_esperanca/.python_env/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/mod

Quantidade de notícias por sentimento predict_final
Negativa    183
Neutra       69
Positiva    312
Name: titulo, dtype: int64 
201 {"statusCode":201,"message":null,"entity":{"id":"fb9489c4-a2b5-4924-a203-d58854b37bb7","title":"SOJA/CEPEA: Preço da soja sobe no BR; exportação de março é recorde para o mês","newsContent":"cotacoes internas soja registrado variacoes positivas ultimos dias resultado avanco preco externo baixa produtividade lavouras argentinas sobretudo maior demanda externa grao brasileiro acordo secex brasil exportou marco dobro volume escoado fevereiro marco totalizando milhoes toneladas maior quantidade soja enviada exterior desde maio recorde marco fonte","publicationDate":"0001-01-01T00:00:00","commodityType":"Soja","source":"CEPEA","positiveSentiment":0.9999999928296575,"neutralSentiment":5.331728583325528E-09,"negativeSentiment":1.8386169435076257E-09,"finalSentiment":"Positiva"},"validationErrors":[]}
201 {"statusCode":201,"message":null,"entity":{"id":"4a529ce7-2

0      None
1      None
2      None
3      None
4      None
       ... 
559    None
560    None
561    None
562    None
563    None
Length: 564, dtype: object